In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Set the path to dataset
import os
home = '/content/drive/MyDrive/2ReadyForFineTuning/CLEVR_HYP/'
imroot = home+'images'
os.chdir(home)

In [3]:
# Prepare dataset in a desired format
prefix = 'clevr_hyp'

import json
import ast
datalen = 0
dumpdict = { "info": {}, "task_type": "Open-Ended", "data_type": "clevr", "license": {}, "data_subtype": prefix, "questions": [] }

with open(home+prefix+'.jsonl') as f:
  data = f.readlines()
  print(len(data))
  global datalen
  datalen = len(data)
  for i in data:
    ijson = json.loads(i)
    imlist = ast.literal_eval(ijson['images'])
    #print(imlist)
    anslist = ast.literal_eval(ijson['answer_choices'])
    try:
      #print(len(anslist))
      assert len(anslist) == 29
    except:
      print(i)
    dumpdict['questions'].append( { "image_id": imroot+"/"+imlist[0].lstrip('./'), "question": ijson['question']+" "+ijson['passage'], "question_id": ijson['qid'], "answer_choices": anslist, "answer_id": int(ijson['answer']) } )

with open('/content/'+prefix+'.jsonl','w+') as w:
  w.write(json.dumps(dumpdict,indent=4))

print(dumpdict)

6250
{'info': {}, 'task_type': 'Open-Ended', 'data_type': 'clevr', 'license': {}, 'data_subtype': 'clevr_hyp', 'questions': [{'image_id': '/content/drive/MyDrive/2ReadyForFineTuning/CLEVR_HYP/images/CLEVR_val_000000.png', 'question': 'How many spheres are there ? Add a blue metal sphere to the scene .', 'question_id': 0, 'answer_choices': ['Yes', 'No', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', 'Red', 'Green', 'Blue', 'Yellow', 'Purple', 'Gray', 'Cyan', 'Brown', 'Cube', 'Sphere', 'Cylinder', 'Small', 'Large', 'Rubber', 'Metal'], 'answer_id': 4}, {'image_id': '/content/drive/MyDrive/2ReadyForFineTuning/CLEVR_HYP/images/CLEVR_val_000001.png', 'question': 'How many metal objects are there ? Add a yellow metal object to the scene .', 'question_id': 1, 'answer_choices': ['Yes', 'No', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', 'Red', 'Green', 'Blue', 'Yellow', 'Purple', 'Gray', 'Cyan', 'Brown', 'Cube', 'Sphere', 'Cylinder', 'Small', 'Large', 'Rubber', 'M

In [4]:
# Load prepared dataset
import json
f = open('/content/'+prefix+'.jsonl','r')
data_questions = json.load(f)
questions = data_questions['questions']

print(data_questions.keys())
print("Number of questions:", len(questions))
print(questions[0])

dict_keys(['info', 'task_type', 'data_type', 'license', 'data_subtype', 'questions'])
Number of questions: 6250
{'image_id': '/content/drive/MyDrive/2ReadyForFineTuning/CLEVR_HYP/images/CLEVR_val_000000.png', 'question': 'How many spheres are there ? Add a blue metal sphere to the scene .', 'question_id': 0, 'answer_choices': ['Yes', 'No', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', 'Red', 'Green', 'Blue', 'Yellow', 'Purple', 'Gray', 'Cyan', 'Brown', 'Cube', 'Sphere', 'Cylinder', 'Small', 'Large', 'Rubber', 'Metal'], 'answer_id': 4}


In [5]:
# Install necessary libraries
!pip install torch
!pip install salesforce-lavis
!pip install transformers

In [6]:
# Import necessary libraries
import torch
from PIL import Image
from lavis.models import load_model_and_preprocess
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from transformers import AutoProcessor, AutoModelForCausalLM
from huggingface_hub import hf_hub_download

/usr/local/lib/python3.10/dist-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/usr/local/lib/python3.10/dist-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


In [7]:
# Check if GPU is available (recommended but not necessary)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [8]:
# Load BLIP model finetuned on VQAv2
model_blip_vqa2, vis_processors_blip_vqa2, txt_processors_blip_vqa2 = load_model_and_preprocess(name="blip_vqa", model_type="vqav2", is_eval=True, device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

100%|██████████| 1.35G/1.35G [00:16<00:00, 87.0MB/s]
/usr/local/lib/python3.10/dist-packages/lavis/models/base_model.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  che

# Batch Run

In [9]:
# Create a batch run script that executes BLIP on an item from the dataset
def batchrun(impath, question, answers):
  raw_image = Image.open(impath).convert("RGB")

  image_blip_vqa2 = vis_processors_blip_vqa2["eval"](raw_image).unsqueeze(0).to(device)
  question_blip_vqa2 = txt_processors_blip_vqa2["eval"](question)
  samples_blip_vqa2 = {"image": image_blip_vqa2, "text_input": question_blip_vqa2}
  bvqa2 = model_blip_vqa2.predict_answers(samples_blip_vqa2, answer_list=answers, inference_method="rank")[0]

  return bvqa2


In [ ]:
# Call batch run script and write results as csv file
import csv
filprefix = "IPQ_BLIP_"+prefix.upper()
with open('/content/'+filprefix+'.csv', 'w+') as k:
  spamwriter = csv.writer(k)
  spamwriter.writerow(["qid","gt_ans","pred_ans_blip","correctness_blip"])
  for exid in range(0,datalen):
    print(exid)
    correctness_blip = 0
    #print(questions[exid]['image_id'], questions[exid]['question'], questions[exid]['answer_choices'])
    pred_ans_blip = batchrun(questions[exid]['image_id'], questions[exid]['question'], [str(i) for i in questions[exid]['answer_choices']])
    gtclass = questions[exid]['answer_choices'][int(questions[exid]['answer_id'])]
    if str(gtclass)==str(pred_ans_blip):
      correctness_blip = 1
    #print(exid,gtclass,pred_ans_blip,correctness_blip)
    spamwriter.writerow([exid,gtclass,pred_ans_blip,correctness_blip])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27